In [1]:
"""
Next Steps:
Try XGBoost
Try t-SNE (https://github.com/danielfrg/tsne)
Build ensamble model
Try TPOT
http://fastml.com/revisiting-numerai/
https://medium.com/jim-fleming/notes-on-the-numerai-ml-competition-14e3d42c19f3
https://github.com/jimfleming/numerai
"""

# Imports
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils
import pandas as pd
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras import losses

Using TensorFlow backend.
/Users/Home/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#import xgboost as xgb

In [3]:
training_df = pd.read_csv("./numerai_datasets/numerai_training_data.csv") #393613 rows (0-393612)
tournament_df = pd.read_csv("./numerai_datasets/numerai_tournament_data.csv") #243484 rows (0-243483)

In [4]:
validation_df = tournament_df.loc[tournament_df['data_type'] == "validation"] # 46362 rows (0-46361)
test_df = tournament_df.loc[tournament_df['data_type'] == "test"] #192605 rows (46362-238966)
live_df = tournament_df.loc[tournament_df['data_type'] == "live"] #4517 rows (238967-243483)
predict_df = tournament_df.loc[tournament_df['data_type'] != "validation"] # 197122 rows (46362-243483)
#tournament_df
#validation_df
#test_df
#live_df
#predict_df

In [5]:
SELECT_SUBSET_PERCENT = 1.0

select_count_train = int(len(training_df)*SELECT_SUBSET_PERCENT)
select_count_valid = int(len(validation_df)*SELECT_SUBSET_PERCENT)

train_X = np.array(training_df.iloc[:select_count_train,3:-1])
train_Y = np.array(training_df.iloc[:select_count_train,-1:])

validation_X = np.array(validation_df.iloc[:select_count_valid,3:-1])
validation_Y = np.array(validation_df.iloc[:select_count_valid,-1:])

In [6]:
# Initialize simple neural network model
model = Sequential()

# Hidden layer 1: 500 neurons, 'relu' activation
model.add(Dense(units=500, input_dim=50))
model.add(Activation('relu'))

# Hidden layer 2: 250 neurons, 'relu' activation
model.add(Dense(units=500))
model.add(Activation('relu'))

# Hidden layer 3: 250 neurons, 'relu' activation
model.add(Dense(units=500))
model.add(Activation('relu'))

# Hidden layer 4: 250 neurons, 'relu' activation
model.add(Dense(units=250))
model.add(Activation('relu'))

# Hidden layer 5: 250 neurons, 'relu' activation
model.add(Dense(units=250))
model.add(Activation('relu'))

# Output layer: 1 neurons, sigmoid
model.add(Dense(units=1))
model.add(Activation('sigmoid'))

optimizer = optimizers.RMSprop()
loss = "binary_crossentropy"
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [7]:
BATCH_SIZE = 50
EPOCHS = 25

In [8]:
model.fit(train_X, train_Y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 314890 samples, validate on 78723 samples
Epoch 1/25
314890/314890 [==============================] - 128s 407us/step - loss: 0.6933 - acc: 0.5030 - val_loss: 0.6951 - val_acc: 0.5023
Epoch 2/25
314890/314890 [==============================] - 110s 350us/step - loss: 0.6929 - acc: 0.5103 - val_loss: 0.6932 - val_acc: 0.5062
Epoch 3/25
314890/314890 [==============================] - 109s 345us/step - loss: 0.6928 - acc: 0.5118 - val_loss: 0.6934 - val_acc: 0.5068
Epoch 4/25
314890/314890 [==============================] - 112s 356us/step - loss: 0.6927 - acc: 0.5136 - val_loss: 0.6933 - val_acc: 0.5042
Epoch 5/25
314890/314890 [==============================] - 115s 365us/step - loss: 0.6926 - acc: 0.5140 - val_loss: 0.6938 - val_acc: 0.5041
Epoch 6/25
314890/314890 [==============================] - 116s 370us/step - loss: 0.6925 - acc: 0.5152 - val_loss: 0.6932 - val_acc: 0.5053
Epoch 7/25
314890/314890 [==============================] - 119s 377us/step - loss: 0.6926 - acc:

In [9]:
loss, acc = model.evaluate(validation_X, validation_Y, batch_size=BATCH_SIZE, verbose=1)

print()
print('Got %.2f%% accuracy' % (acc * 100.))
print('Loss: ', loss)

46362/46362 [==============================] - 4s 84us/step

Got 51.33% accuracy
Loss:  0.69276875501


In [10]:
predicted_df = model.predict(predict_df.iloc[:,3:-1])

In [17]:
predicted_df
#s1 = pd.Series(predicted_df, name='probability')
#pd.DataFrame(predicted_df)

array([[ 0.50881368],
       [ 0.49623519],
       [ 0.47298211],
       ..., 
       [ 0.45565328],
       [ 0.51859486],
       [ 0.49515283]], dtype=float32)

In [19]:
predict_df.iloc[:,:1].values

array([['n76c75f57cec64cb'],
       ['ncc70463e0b6f43c'],
       ['nbc4e5adb0ebd41e'],
       ..., 
       ['nf4f724fd48bb4cf'],
       ['n45f1d8e9ab8d440'],
       ['nf2c8618627354fe']], dtype=object)

In [25]:
final_predictions_df = pd.DataFrame(np.concatenate((predict_df.iloc[:,:1].values, predicted_df), axis=1))

In [26]:
csv_path = 'predictions_stenger.csv'

In [27]:
final_predictions_df.to_csv(csv_path, index=None)

In [28]:
print('Saved: {}'.format(csv_path))

Saved: predictions_stenger.csv
